In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [ ]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

### Let's divide the training and test set into two partitions

In [ ]:
# Your code
# Separate Features and Target
X = data['text']
y = data['label']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Data Preprocessing

In [ ]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [ ]:
# Your code
# Text Preprocessing
cleaned_texts = []

for text in X_train.values:
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    for script in soup(["script", "style"]):
        script.extract()
    cleaned_html = re.sub(r"<!--.*?-->", "", str(soup), flags=re.DOTALL)
    text = re.sub(r"<[^>]+>", " ", cleaned_html)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
# Your code
    # Remove special characters, numbers, and single characters
text = re.sub(r"[^a-zA-Z\s]", " ", text)  
text = re.sub(r"\b[a-zA-Z]\b", " ", text)  
text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)  
text = re.sub(r"\s+", " ", text).strip()
    
    # Lowercase conversion
text = text.lower()

## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
# Your code

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Your code
    # Tokenization
words = word_tokenize(text)
    
    # Remove stop words
words = [word for word in words if word not in stopwords.words('english')]

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# Your code
    # Lemmatization
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word) for word in words]

## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code
    # Lemmatization
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word) for word in words]

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code
# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(cleaned_texts)

# Basic Feature Engineering
money_symbols = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])


## And the Train a Classifier?

In [ ]:
# Your code
# Adding Features
data['money_mark'] = data['text'].str.contains(money_symbols) * 1
data['suspicious_words'] = data['text'].str.contains(suspicious_words) * 1
data['text_len'] = data['text'].apply(lambda x: len(x))

# Display updated dataframe
print(data.head())

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code
# Spam vs Ham Word Frequency
spam_texts = data[data["label"] == "spam"]["text"]
ham_texts = data[data["label"] == "ham"]["text"]

spam_words = " ".join(spam_texts).split()
ham_words = " ".join(ham_texts).split()

spam_word_counts = Counter(spam_words)
ham_word_counts = Counter(ham_words)

# Top 10 Spam & Ham Words
top_spam_words = spam_word_counts.most_common(10)
top_ham_words = ham_word_counts.most_common(10)

# Print Top Words
print("Top Spam Words:", top_spam_words)
print("Top Ham Words:", top_ham_words)